# Sprint14課題 ディープラーニングフレームワーク1

## 【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。

（例）

- 重みを初期化する必要があった
- エポックのループが必要だった


それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。

- レイヤー別に種類を指定し、インスタンスを生成する必要があった
- 各層に対して次元(shape)を合わせる必要があった
- 学習率を引数にて指定する必要があった
- 出力サイズを計算する必要があった
- 活性化関数(ReLU, sigmoid, tanh)を指定する必要があった
- 最適化手法(SGD, AdaGrad)を指定する必要があった
- 重みの初期化手法(He, Xavier)を選択する必要があった
- 全結合層に対しては、平滑化した入力データが必要だった
- ミニバッチサイズを指定する必要があった
- 全結合層に対しては、レイヤー別にノード数を指定する必要があった

### データセットの用意
以前から使用しているIrisデータセットを使用します。以下のサンプルコードではIris.csvが同じ階層にある想定です。

[Iris Species](https://www.kaggle.com/uciml/iris/data)

目的変数はSpeciesですが、3種類ある中から以下の2種類のみを取り出して使用します。

- Iris-versicolor
- Iris-virginica

## 【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。

それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

サンプルコード

＊バージョン1.5から1.13の間で動作を確認済みです。

In [16]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """

    def __init__(self, X, y, batch_size=10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)

    def __len__(self):
        return self._stop

    def __getitem__(self, item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]

    def __iter__(self):
        self._counter = 0
        return self

    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [17]:
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(
        layer_2, weights['w3']) + biases['b3']  # tf.addと+は等価である
    return layer_output

In [18]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

# データセットの読み込み
dataset_path = "Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor") |
        (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm",
               "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y == 'Iris-versicolor'] = 0
y[y == 'Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=0)


# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 20

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

# ネットワーク構造の読み込み
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={
                                 X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= total_batch
        total_acc /= total_batch
        val_loss, val_acc = sess.run(
            [loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(
            epoch, total_loss, val_loss, total_acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 73.5804, val_loss : 46.8297, acc : 0.521, val_acc : 0.625
Epoch 1, loss : 34.3071, val_loss : 30.2668, acc : 0.579, val_acc : 0.375
Epoch 2, loss : 17.0055, val_loss : 3.3999, acc : 0.621, val_acc : 0.812
Epoch 3, loss : 7.4064, val_loss : 2.7802, acc : 0.700, val_acc : 0.750
Epoch 4, loss : 2.9025, val_loss : 1.5643, acc : 0.829, val_acc : 0.875
Epoch 5, loss : 2.0979, val_loss : 2.2962, acc : 0.814, val_acc : 0.812
Epoch 6, loss : 2.1418, val_loss : 0.6701, acc : 0.871, val_acc : 0.938
Epoch 7, loss : 1.1022, val_loss : 0.0073, acc : 0.900, val_acc : 1.000
Epoch 8, loss : 0.7134, val_loss : 0.5347, acc : 0.957, val_acc : 0.938
Epoch 9, loss : 0.4282, val_loss : 0.2739, acc : 0.914, val_acc : 0.938
Epoch 10, loss : 0.8457, val_loss : 0.5538, acc : 0.943, val_acc : 0.938
Epoch 11, loss : 0.5477, val_loss : 0.5661, acc : 0.914, val_acc : 0.875
Epoch 12, loss : 0.8901, val_loss : 0.6881, acc : 0.943, val_acc : 0.938
Epoch 13, loss : 0.7505, val_loss : 0.6943, acc : 0.900,

- 重み  
*tf.random_normal*を用いて正規分布からランダムな値を計算して初期化をし、*tf.Valiable*へデータフローグラフ構築時に値を持たせる。


- エポック  
スクラッチ同様、指定したエポック数に応じてループがされている


- レイヤー（層）  
レイヤーごとに層の種類、活性化関数(ex : tf.relu)を指定する


- 学習率  
スクラッチ同様、ハイパーパラメータとして指定


- 最適化手法  
*tf.train.AdamOptimizer*にてAdamアルゴリズムを指定し、minimizeにて最小化させる目的関数(tf.reduce_meanにて指定)を設定する


- ミニバッチ  
スクラッチ同様、ハイパーパラメータとして指定


- ノード数  
スクラッチ同様、ハイパーパラメータとして指定。重みとバイアスの宣言時に各レイヤーごとのノード数に応じて乱数初期化を行う。


## 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
[Iris Species](https://www.kaggle.com/uciml/iris/data)

サンプルコードと同じくこの中のtrain.csvを使用してください。目的変数はSpeciesに含まれる3種類全てを使います。

2クラスの分類と3クラス以上の分類の違いを考慮してください。

それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。

ヒント

以下の2箇所は2クラス分類特有の処理です。

```python
oss_op = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
```

```python
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
```

メソッドは以下のように公式ドキュメントを確認してください。

[tf.nn.sigmoid_cross_entropy_with_logits  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/nn/sigmoid_cross_entropy_with_logits)

[tf.math.sign  |  TensorFlow](https://www.tensorflow.org/api_docs/python/tf/math/sign)

＊tf.signとtf.math.signは同じ働きをします。

In [19]:
from sklearn.preprocessing import OneHotEncoder


def onehot(y):
    """
    多クラス分類を行う際のone-hot表現に変換

    Parameters
    ----------
    y : 次の形のndarray, shape (n_samples, )
        サンプル

    Returns
    -------
    y_one_hot : 次の形のndarray, shape (n_samples, n_classes)
        推定結果
    """
    enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
    y_one_hot = enc.fit_transform(y[:, np.newaxis])

    return y_one_hot

In [20]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを3値分類する
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

# データセットの読み込み
dataset_path = "Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm",
               "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y == 'Iris-versicolor'] = 0
y[y == 'Iris-virginica'] = 1
y[y == 'Iris-setosa'] = 2  # 追加
y = y.astype(np.int)[:, np.newaxis]

# 多クラス分類のため、ワンホットエンコーディングする
y = onehot(y.reshape(-1))

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=0)


# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

# ネットワーク構造の読み込み
logits = example_net(X)

# 目的関数(ソフトマックスクロスエントロピー)
loss_op = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))

# 最適化手法(GradientDescentOptimizer : 最急降下法)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
# 多クラス分類のため、最大値のインデックスが一致しているかで推定する
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(tf.nn.softmax(logits), 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={
                                 X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= total_batch
        total_acc /= total_batch
        val_loss, val_acc = sess.run(
            [loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(
            epoch, total_loss, val_loss, total_acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 46.3917, val_loss : 46.9633, acc : 0.467, val_acc : 0.583
Epoch 1, loss : 14.7302, val_loss : 41.5780, acc : 0.657, val_acc : 0.583
Epoch 2, loss : 12.4204, val_loss : 59.6991, acc : 0.617, val_acc : 0.292
Epoch 3, loss : 11.7407, val_loss : 3.8696, acc : 0.720, val_acc : 0.583
Epoch 4, loss : 12.0987, val_loss : 33.0661, acc : 0.743, val_acc : 0.583
Epoch 5, loss : 10.2755, val_loss : 44.9036, acc : 0.767, val_acc : 0.583
Epoch 6, loss : 7.5043, val_loss : 34.3293, acc : 0.737, val_acc : 0.583
Epoch 7, loss : 5.4504, val_loss : 27.5288, acc : 0.797, val_acc : 0.583
Epoch 8, loss : 3.2603, val_loss : 2.5207, acc : 0.870, val_acc : 0.833
Epoch 9, loss : 2.1279, val_loss : 2.7349, acc : 0.860, val_acc : 0.875
test_acc : 0.867


## 【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。

[House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。

分類問題と回帰問題の違いを考慮してください。

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn import preprocessing

In [22]:
df = pd.read_csv('train.csv')

In [23]:
# データの確認
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [24]:
# 目的変数、説明変数の抽出
X = df.loc[:, ['GrLivArea', 'YearBuilt']].values
y = df.loc[:, 'SalePrice'].values

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=0)

# 説明変数データの標準化
ss_x = preprocessing.StandardScaler()
ss_x.fit(X_train.astype(float))
X_train_std = ss_x.transform(X_train.astype(float))
X_val_std = ss_x.transform(X_val.astype(float))
X_test_std = ss_x.transform(X_test.astype(float))

# 目的変数データの標準化
ss_y = preprocessing.StandardScaler()
ss_y.fit(y_train.reshape(-1, 1).astype(float))
y_train_std = ss_y.transform(y_train.reshape(-1, 1).astype(float))
y_val_std = ss_y.transform(y_val.reshape(-1, 1).astype(float))
y_test_std = ss_y.transform(y_test.reshape(-1, 1).astype(float))

# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 30

n_hidden1 = 10
n_hidden2 = 5
n_input = X_train_std.shape[1]
n_samples = X_train_std.shape[0]
# 回帰問題のため、クラス数は1とする
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train_std, y_train_std, batch_size)

# ネットワーク構造の読み込み
logits = example_net(X)

# 目的関数(平均二乗誤差)
loss_op = tf.reduce_mean(tf.square(logits - Y))
# 最適化手法
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 回帰のため、精度は省略

init = tf.global_variables_initializer()

# 計算グラフの実行

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train_std.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={
                     X: mini_batch_x, Y: mini_batch_y.reshape(-1, 1)})
            loss = sess.run(loss_op, feed_dict={
                            X: mini_batch_x, Y: mini_batch_y.reshape(-1, 1)})
            total_loss += loss
        total_loss /= total_batch
        val_loss = sess.run(loss_op, feed_dict={
                            X: X_val_std, Y: y_val_std.reshape(-1, 1)})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}".format(
            epoch, total_loss, val_loss))
    test_loss = sess.run(loss_op, feed_dict={
                         X: X_test_std, Y: y_test_std.reshape(-1, 1)})
    print("test_loss : {:.3f}".format(test_loss))

Epoch 0, loss : 1.1604, val_loss : 0.5633
Epoch 1, loss : 0.6614, val_loss : 0.5062
Epoch 2, loss : 0.5940, val_loss : 0.4787
Epoch 3, loss : 0.5527, val_loss : 0.4595
Epoch 4, loss : 0.5229, val_loss : 0.4449
Epoch 5, loss : 0.5002, val_loss : 0.4332
Epoch 6, loss : 0.4821, val_loss : 0.4235
Epoch 7, loss : 0.4673, val_loss : 0.4143
Epoch 8, loss : 0.4547, val_loss : 0.4062
Epoch 9, loss : 0.4444, val_loss : 0.3980
Epoch 10, loss : 0.4347, val_loss : 0.3909
Epoch 11, loss : 0.4261, val_loss : 0.3848
Epoch 12, loss : 0.4182, val_loss : 0.3783
Epoch 13, loss : 0.4112, val_loss : 0.3726
Epoch 14, loss : 0.4042, val_loss : 0.3665
Epoch 15, loss : 0.3974, val_loss : 0.3616
Epoch 16, loss : 0.3913, val_loss : 0.3573
Epoch 17, loss : 0.3857, val_loss : 0.3534
Epoch 18, loss : 0.3806, val_loss : 0.3499
Epoch 19, loss : 0.3760, val_loss : 0.3463
Epoch 20, loss : 0.3718, val_loss : 0.3428
Epoch 21, loss : 0.3677, val_loss : 0.3391
Epoch 22, loss : 0.3639, val_loss : 0.3358
Epoch 23, loss : 0.36

## 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。

3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。

スクラッチで実装したモデルの再現を目指してください。

In [25]:
# ライブラリのインポート
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [26]:
# mnistデータを読み込み
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [27]:
# (n_samples, n_channels, height, width)のNCHW
X_train = X_train.reshape(60000, 1, 28, 28)
X_test = X_test.reshape(10000, 1, 28, 28)

In [28]:
# float型へ
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)

# 正規化
X_train /= 255
X_test /= 255
print(X_train.max())  # 1.0
print(X_train.min())  # 0.0

# 平滑化
X_train = X_train.reshape(-1, 1*28*28)
X_test = X_test.reshape(-1, 1*28*28)

1.0
0.0


In [29]:
# データ分割
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2)
print(X_train.shape)
print(X_val.shape)


# ワンホットを追加
y_train = onehot(y_train.reshape(-1))
y_val = onehot(y_val.reshape(-1))
y_test = onehot(y_test.reshape(-1))

(48000, 784)
(12000, 784)


In [30]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
# 0〜9の10クラス
n_classes = 10

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

# ネットワーク構造の読み込み
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
# 最適化手法(Admaアルゴリズムの方が精度良さそう)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(tf.nn.softmax(logits), 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={
                                 X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= total_batch
        total_acc /= total_batch
        val_loss, val_acc = sess.run(
            [loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(
            epoch, total_loss, val_loss, total_acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 5.1805, val_loss : 1.2913, acc : 0.746, val_acc : 0.685
Epoch 1, loss : 0.8403, val_loss : 1.1717, acc : 0.760, val_acc : 0.780
Epoch 2, loss : 0.5672, val_loss : 0.4544, acc : 0.839, val_acc : 0.886
Epoch 3, loss : 0.3424, val_loss : 0.3402, acc : 0.905, val_acc : 0.909
Epoch 4, loss : 0.2838, val_loss : 0.3359, acc : 0.924, val_acc : 0.915
Epoch 5, loss : 0.2568, val_loss : 0.3589, acc : 0.932, val_acc : 0.914
Epoch 6, loss : 0.2380, val_loss : 0.3314, acc : 0.937, val_acc : 0.923
Epoch 7, loss : 0.2361, val_loss : 0.3290, acc : 0.938, val_acc : 0.923
Epoch 8, loss : 0.2289, val_loss : 0.3316, acc : 0.939, val_acc : 0.925
Epoch 9, loss : 0.2239, val_loss : 0.3665, acc : 0.942, val_acc : 0.925
test_acc : 0.930
